In [ ]:
cd ctf_challs/LW5

In [1]:
from pwn import *
from ast import literal_eval

context.log_level = "critical"

#sh = process(["sage", "chal.sage"])
sh = remote("62.234.62.107", 30617)
#sh = remote("localhost", 10007)
sh.recvuntil(b"your error plz :)")
sh.sendline(str([259832, 985692, 628744, 767011, 478513]).encode())

gift = literal_eval(sh.recvline().strip().decode()[4:])
enc = literal_eval(sh.recvline().strip().decode()[4:])

In [2]:
###################################################################################### another method
p = 1048583
E = [259832, 985692, 628744, 767011, 478513]
m, n = 90, 56
A = Matrix(Zmod(p), m, n, gift[:m*n])
b = vector(Zmod(p), gift[m*n:])
nums = 5

from Crypto.Util.number import *

t = 2
e_ = vector(ZZ, [i - E[0] for i in E][1:])

L = block_matrix(ZZ, [
    [1, (Matrix(ZZ, [_ * inverse(e_[-1], p) for _ in e_[:-1]])).T],
    [0, p]
])
L = L.BKZ()

########################################################### handle with Lattice
U = []
for _ in range(nums-t-1):
    U.append(L[_].list()[:-1] + [-L[_].list()[-1]])
U = Matrix(ZZ, U)
K = Matrix(ZZ, U.right_kernel().basis())
print(K.dimensions())
X = (K.BKZ()).T
print(X)
X = Matrix(Zmod(p), X)
g = X.solve_right(e_)

print(g)
########################################################### handle with Lattice

d = m

A_ker = A[:d].left_kernel().matrix()
Fp = Zmod(p)
V = Fp["".join(["x"+str(i)+", " for i in range(d)]) + "".join(["y"+str(i)+", " for i in range(d)]).strip(", ")]
x, y = vector(V.gens()[:d]), vector(V.gens()[d:2*d])

XY = (Matrix(x).stack(Matrix(y))).T
ve = vector(Zmod(p), g.list())
XYe = XY*ve

polys = A_ker*XYe - A_ker*(b[:d]-vector([E[0]]*d))
M, V = Sequence(polys).coefficients_monomials()
M = M.T.dense_matrix()

(2, 4)
[ 1  0]
[ 0  2]
[-1  1]
[ 2 -1]
(725860, 184456)


In [3]:
from subprocess import check_output
from re import findall

def blaster(M, block_size=40, bkz_tour=5):
    z = "[[" + "]\n[".join(" ".join(map(str, row)) for row in M) + "]]"
    open("Lattice_test", 'w').write(z+'\n')

    ######################################## if need verbose, add "-v"
    ret = check_output([
        "python3", "blaster/src/app.py",
        "-i", "Lattice_test", "-b" + str(block_size), "-t" + str(bkz_tour), "-v"
        ])
    return matrix(M.nrows(), M.ncols(), map(int, findall(b"-?\\d+", ret)))

In [4]:
#################################################################### use reduction
M1, M2 = M[:d-n], M[d-n:]

if(1):
    L = block_matrix(ZZ,[
        [1, -M2*M1^(-1)],
        [0, p]
    ])

print(L.dimensions())
L = blaster(L, 63, 10)
#L = L.BKZ(block_size=10)
print(L[0])
print(float(L[0].norm()))

(181, 181)


E[∥b₁∥] ~ 667.50 < 1048583 (GH: λ₁ ~ 44.01)
......................................
BKZ(β: 63,t: 1/10, o:   0): slope=-0.043671, rhf=1.013994.................
BKZ(β: 63,t: 1/10, o:   2): slope=-0.042644, rhf=1.013994...........
BKZ(β: 63,t: 1/10, o:   4): slope=-0.042165, rhf=1.013994.....
BKZ(β: 63,t: 1/10, o:   6): slope=-0.041993, rhf=1.013139..
BKZ(β: 63,t: 1/10, o:   8): slope=-0.041804, rhf=1.013139....
BKZ(β: 63,t: 1/10, o:  10): slope=-0.041680, rhf=1.013139..
BKZ(β: 63,t: 1/10, o:  12): slope=-0.041577, rhf=1.013139...........
BKZ(β: 63,t: 1/10, o:  14): slope=-0.041276, rhf=1.013139....
BKZ(β: 63,t: 1/10, o:  16): slope=-0.041130, rhf=1.013139..
BKZ(β: 63,t: 1/10, o:  18): slope=-0.040994, rhf=1.013139..
BKZ(β: 63,t: 1/10, o:  20): slope=-0.040817, rhf=1.013139..
BKZ(β: 63,t: 1/10, o:  22): slope=-0.040759, rhf=1.013139...
BKZ(β: 63,t: 1/10, o:  24): slope=-0.040671, rhf=1.013139.....
BKZ(β: 63,t: 1/10, o:  26): slope=-0.040625, rhf=1.013139...
BKZ(β: 63,t: 1/10, o:  28): slop

(-1, -1, 0, 0, -1, 0, 0, 0, -2, 0, -2, 1, -1, 0, -1, 0, -2, -1, 0, -1, -1, -2, 0, -2, -1, -1, 1, -1, 0, -2, 0, 0, 0, 0, -1, -2, -1, -2, 1, -1, 0, 1, 0, 0, 0, -2, -1, 0, -2, 0, 0, 0, -1, 0, 0, 0, 1, -1, 0, 0, 0, 0, -1, -2, 0, 1, 0, 0, 0, 1, -1, 1, -2, -1, 0, 0, 1, -2, 0, -1, 1, 1, -2, 0, 0, -2, -2, -1, 0, 0, 0, 0, 0, -2, 0, 0, 0, 0, 1, 0, 1, -1, 0, 0, 0, 0, 1, 0, -2, 0, 0, 1, 0, 1, 0, 0, -1, 0, -2, 1, 0, -2, -2, 0, 0, 1, 0, 1, -1, 0, -2, -1, 0, -2, 0, 1, 0, 0, 1, -2, 0, 0, 0, -2, 0, 0, -1, -2, 1, 0, 0, 0, 0, 1, 0, 0, -2, -1, 0, 0, -2, 1, -2, 0, 1, 0, -1, -2, 0, -1, 1, -2, -2, 0, 0, -1, 0, 0, 1, 0, 0)
13.638181696985855



BKZ(β: 63,t:10/10, o: 120): slope=-0.033786, rhf=1.000049.
Iterations: 1856
t_{QR-decomp. }=     2.481s
t_{LLL-red.   }=     0.910s
t_{BKZ-red.   }=   487.989s
t_{Seysen-red.}=     3.186s
t_{Matrix-mul.}=     1.751s


In [6]:
res = vector(ZZ, L[0]).list()
nums = m
res = res[nums+n+1:] + res[:nums+n]

recover_e = []
for i in range(m):
    if(res[i] == 0 and res[i+nums] == 0):
        recover_e.append(E[0])
    elif((res[i] == -1 and res[i+nums] == 0) or (res[i] == 1 and res[i+nums] == 0)):
        recover_e.append(E[1])
    elif((res[i] == 0 and res[i+nums] == 2) or (res[i] == 0 and res[i+nums] == -2)):
        recover_e.append(E[2])
    elif((res[i] == 1 and res[i+nums] == -1) or (res[i] == -1 and res[i+nums] == 1)):
        recover_e.append(E[3])
    elif((res[i] == 2 and res[i+nums] == -1) or (res[i] == -2 and res[i+nums] == 1)):
        recover_e.append(E[4])
    else:
        print(res[i], res[i+nums])
        recover_e.append("?")
print(E)
print(recover_e)

[259832, 985692, 628744, 767011, 478513]
[478513, 767011, 259832, 259832, 259832, 259832, 767011, 628744, 259832, 478513, 985692, 259832, 259832, 478513, 767011, 478513, 628744, 767011, 259832, 985692, 478513, 628744, 985692, 767011, 478513, 478513, 628744, 259832, 985692, 628744, 628744, 767011, 259832, 259832, 985692, 985692, 259832, 628744, 985692, 259832, 259832, 259832, 478513, 259832, 478513, 767011, 985692, 259832, 985692, 259832, 478513, 985692, 628744, 985692, 985692, 478513, 259832, 478513, 985692, 985692, 767011, 985692, 628744, 478513, 259832, 628744, 628744, 259832, 985692, 478513, 985692, 478513, 767011, 985692, 628744, 767011, 259832, 628744, 259832, 478513, 985692, 259832, 478513, 628744, 259832, 259832, 985692, 628744, 259832, 259832]


In [7]:
from Crypto.Cipher import AES
from hashlib import md5

s = A[:nums].solve_right(b[:nums] - vector(GF(p), recover_e))
flag = AES.new(key=md5((str(s)).encode()).digest(), nonce=b"Tiffany", mode=AES.MODE_CTR).decrypt(enc)
print(flag)


#WMCTF{F1nD_@_BETTER_L1n3aR_b@SIS!}

b'WMCTF{F1nD_@_BETTER_L1n3aR_b@SIS!}'


In [ ]:
sh.close()